# 📘 Notebook 1 — Basic Media: Camera & Audio

> 🎯 **Goal**: Learn to capture images, record audio, and play sounds - Make Reachy see and hear!

---

## 0. What You Will Learn

By the end of this notebook, you will be able to:

* 📸 Capture images from Reachy's camera
* 🎬 Display real-time video feed
* 🎤 Record audio from the microphone array
* 🔊 Play sounds through the speaker
* 💾 Save and load media files
* 🤖 Make Reachy feel more "alive" with audiovisual feedback

**Duration:** 20 minutes

**Note:** This notebook requires the camera and microphone to be functional. Make sure NOT to use `media_backend="no_media"` this time!

---

## 1. Setup

First, let's import the necessary libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
from reachy_mini import ReachyMini
from reachy_mini.utils import create_head_pose

# For audio processing
try:
    import soundfile as sf
    import scipy.signal
    print("✓ All libraries imported successfully!")
except ImportError as e:
    print(f"⚠ Missing library: {e}")
    print("Install with: pip install soundfile scipy")

---

## Part 1: Camera Basics 📸

### 2. Capturing Your First Image

Let's capture a single frame from Reachy's camera!

In [ ]:
# Connect WITH media enabled (notice we don't use media_backend="no_media")
with ReachyMini() as mini:
    print("Capturing image...")
    
    # Get a frame from the camera
    frame = mini.media.get_frame()
    
    # Check if we got a valid frame
    if frame is not None:
        print(f"✓ Image captured!")
        print(f"  Resolution: {frame.shape[1]}x{frame.shape[0]}")
        print(f"  Channels: {frame.shape[2]} (BGR format)")
        print(f"  Data type: {frame.dtype}")
    else:
        print("✗ Failed to capture image")

**What You Just Did:**

* **`mini.media.get_frame()`**: Returns a numpy array containing the image
* **Format**: OpenCV format (BGR color order, not RGB!)
* **Shape**: `(height, width, channels)` - typically `(480, 640, 3)` or similar
* **Data type**: `uint8` (0-255 for each pixel)

**Important:** The frame is in **BGR format** (Blue-Green-Red), not RGB! This is OpenCV's default. We'll handle the conversion when displaying.

### 3. Displaying the Image

Let's display the captured image using matplotlib.

In [ ]:
with ReachyMini() as mini:
    # Capture frame
    frame = mini.media.get_frame()
    
    if frame is not None:
        # Convert BGR to RGB for matplotlib
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Display
        plt.figure(figsize=(10, 8))
        plt.imshow(frame_rgb)
        plt.title("Reachy's View")
        plt.axis('off')
        plt.show()
    else:
        print("No frame available")

### 4. Saving an Image to Disk

Let's save the captured image as a file.

In [ ]:
with ReachyMini() as mini:
    frame = mini.media.get_frame()
    
    if frame is not None:
        # Save with OpenCV (no conversion needed - it expects BGR)
        filename = f"reachy_photo_{int(time.time())}.jpg"
        cv2.imwrite(filename, frame)
        print(f"✓ Image saved as: {filename}")
    else:
        print("✗ No frame to save")

### 5. Live Camera Feed

Let's capture multiple frames and create a simple "video" display.

**Note:** In a Jupyter notebook, this will update the image several times. Press the stop button (■) to interrupt.

In [ ]:
with ReachyMini() as mini:
    print("Starting live feed... (Press Stop button to interrupt)")
    
    try:
        for i in range(20):  # Capture 20 frames
            frame = mini.media.get_frame()
            
            if frame is not None:
                # Convert and display
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                plt.figure(figsize=(8, 6))
                plt.imshow(frame_rgb)
                plt.title(f"Live Feed - Frame {i+1}/20")
                plt.axis('off')
                plt.show()
                
                time.sleep(0.5)  # 2 FPS for display
            else:
                print(f"Frame {i+1}: No data")
                
    except KeyboardInterrupt:
        print("\nLive feed stopped")
    
    print("Done!")

**Performance Note:**
* The camera typically runs at 30 FPS
* `get_frame()` returns the latest available frame
* For real-time applications, you can call `get_frame()` in a loop at your desired rate
* Displaying in Jupyter is slow - in a real application, you'd use OpenCV's `cv2.imshow()`

---

## Part 2: Audio Basics 🎤🔊

### 6. Understanding Reachy's Audio System

Reachy Mini has:
* **Microphone Array**: A ReSpeaker 4-mic array (we'll explore Direction of Arrival in a later notebook)
* **Speaker**: For playing sounds
* **Sample Rate**: 16 kHz (16,000 samples per second)
* **Format**: Mono or stereo audio depending on operation

Let's check the audio configuration:

In [ ]:
with ReachyMini() as mini:
    input_rate = mini.media.get_input_audio_samplerate()
    output_rate = mini.media.get_output_audio_samplerate()
    
    print(f"Input (Microphone) Sample Rate: {input_rate} Hz")
    print(f"Output (Speaker) Sample Rate: {output_rate} Hz")

### 7. Recording Audio

Let's record 3 seconds of audio from the microphone.

**Try this:** Say something or clap while running this cell!

In [ ]:
RECORD_DURATION = 3  # seconds

with ReachyMini() as mini:
    print(f"Recording for {RECORD_DURATION} seconds...")
    print("🎤 Say something or make a sound!")
    
    # Start recording
    mini.media.start_recording()
    
    audio_samples = []
    start_time = time.time()
    
    # Collect audio samples
    while time.time() - start_time < RECORD_DURATION:
        sample = mini.media.get_audio_sample()
        
        if sample is not None:
            audio_samples.append(sample)
            print(f"\rRecording... {time.time() - start_time:.1f}s", end="")
        
        time.sleep(0.1)  # Sample at ~10 Hz
    
    # Stop recording
    mini.media.stop_recording()
    
    print(f"\n✓ Recording complete!")
    print(f"  Captured {len(audio_samples)} samples")
    
    # Concatenate all samples into one array
    if audio_samples:
        audio_data = np.concatenate(audio_samples, axis=0)
        print(f"  Total audio shape: {audio_data.shape}")
        print(f"  Duration: {len(audio_data) / mini.media.get_input_audio_samplerate():.2f} seconds")
    else:
        print("  ⚠ No audio data captured")
        audio_data = None

**What Just Happened:**

* **`start_recording()`**: Enables audio capture from the microphone
* **`get_audio_sample()`**: Returns a chunk of audio data (numpy array)
* **`stop_recording()`**: Disables audio capture
* **Data format**: Numpy array of float32 values
* **Sample chunks**: Each call to `get_audio_sample()` returns a small chunk (typically 0.1-0.2 seconds worth)

### 8. Visualizing the Audio

Let's plot the waveform of what we just recorded.

In [ ]:
if audio_data is not None and len(audio_data) > 0:
    # Create time axis
    sample_rate = 16000  # 16 kHz
    time_axis = np.arange(len(audio_data)) / sample_rate
    
    # Plot waveform
    plt.figure(figsize=(14, 4))
    
    # Handle stereo (2 channels) or mono (1 channel)
    if audio_data.ndim > 1:
        # Stereo: plot first channel
        plt.plot(time_axis, audio_data[:, 0])
        plt.title("Recorded Audio Waveform (Channel 1)")
    else:
        # Mono
        plt.plot(time_axis, audio_data)
        plt.title("Recorded Audio Waveform")
    
    plt.xlabel("Time (seconds)")
    plt.ylabel("Amplitude")
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("No audio data to visualize. Run the recording cell first!")

### 9. Saving Audio to File

Let's save the recorded audio as a WAV file.

In [ ]:
if audio_data is not None and len(audio_data) > 0:
    filename = f"reachy_recording_{int(time.time())}.wav"
    sample_rate = 16000
    
    # Save with soundfile
    sf.write(filename, audio_data, sample_rate)
    print(f"✓ Audio saved as: {filename}")
    print(f"  You can play it with any media player!")
else:
    print("No audio data to save. Run the recording cell first!")

### 10. Playing Audio

Now let's play audio through Reachy's speaker! We'll create a simple helper function.

In [ ]:
def play_audio_file(mini, audio_file_path):
    """Helper function to play an audio file through Reachy's speaker.
    
    Args:
        mini: ReachyMini instance
        audio_file_path: Path to WAV file
    """
    # Load audio file
    data, samplerate_in = sf.read(audio_file_path, dtype='float32')
    
    # Resample if needed
    target_rate = mini.media.get_output_audio_samplerate()
    if samplerate_in != target_rate:
        print(f"Resampling from {samplerate_in} Hz to {target_rate} Hz...")
        num_samples = int(len(data) * (target_rate / samplerate_in))
        data = scipy.signal.resample(data, num_samples)
    
    # Convert stereo to mono if needed
    if data.ndim > 1:
        print("Converting stereo to mono...")
        data = np.mean(data, axis=1)
    
    # Start playback
    mini.media.start_playing()
    print("🔊 Playing audio...")
    
    # Push samples in chunks
    chunk_size = 1024
    for i in range(0, len(data), chunk_size):
        chunk = data[i:i + chunk_size]
        mini.media.push_audio_sample(chunk)
    
    # Wait for playback to finish
    time.sleep(len(data) / target_rate)
    mini.media.stop_playing()
    print("✓ Playback complete!")

print("Helper function defined!")

### 11. Play Back What We Recorded

Let's play back the audio we just recorded!

In [ ]:
# Make sure you've run the recording and saving cells first!
# Update this filename to match your saved recording
audio_file = "reachy_recording_1234567890.wav"  # Update with actual filename

# Or use the last saved file from the variables
if 'filename' in dir():
    audio_file = filename
    print(f"Using file: {audio_file}")

try:
    with ReachyMini() as mini:
        play_audio_file(mini, audio_file)
except FileNotFoundError:
    print(f"⚠ File not found: {audio_file}")
    print("Make sure to run the recording and saving cells first!")

### 12. Generate and Play a Tone

Let's create a simple beep sound programmatically!

In [ ]:
def generate_beep(frequency=440, duration=0.5, sample_rate=16000):
    """Generate a simple sine wave tone.
    
    Args:
        frequency: Frequency in Hz (440 = A note)
        duration: Duration in seconds
        sample_rate: Sample rate in Hz
    
    Returns:
        numpy array of audio samples
    """
    t = np.linspace(0, duration, int(sample_rate * duration))
    tone = 0.3 * np.sin(2 * np.pi * frequency * t)  # 0.3 = volume
    
    # Add fade in/out to avoid clicks
    fade_samples = int(sample_rate * 0.01)  # 10ms fade
    fade_in = np.linspace(0, 1, fade_samples)
    fade_out = np.linspace(1, 0, fade_samples)
    tone[:fade_samples] *= fade_in
    tone[-fade_samples:] *= fade_out
    
    return tone.astype(np.float32)

# Generate a beep
beep = generate_beep(frequency=880, duration=0.3)  # High A note

with ReachyMini() as mini:
    mini.media.start_playing()
    print("🔔 Beep!")
    
    # Push the beep
    chunk_size = 1024
    for i in range(0, len(beep), chunk_size):
        chunk = beep[i:i + chunk_size]
        mini.media.push_audio_sample(chunk)
    
    time.sleep(0.5)
    mini.media.stop_playing()
    print("Done!")

---

## 13. Combining Media and Motion

Let's make Reachy more interactive by combining what we've learned!

In [ ]:
# "Photo taking" behavior: Beep, look at camera, take photo, beep again
with ReachyMini() as mini:
    print("📸 Photo session starting...")
    
    # Move to neutral
    mini.goto_target(
        head=create_head_pose(),
        antennas=[0.0, 0.0],
        duration=1.0
    )
    
    # Countdown with beeps and head movements
    for i in [3, 2, 1]:
        print(f"  {i}...")
        
        # Beep
        beep = generate_beep(frequency=440 + i*100, duration=0.2)
        mini.media.start_playing()
        for j in range(0, len(beep), 1024):
            mini.media.push_audio_sample(beep[j:j+1024])
        mini.media.stop_playing()
        
        # Wiggle antennas
        mini.goto_target(antennas=[0.3, -0.3], duration=0.3)
        mini.goto_target(antennas=[0.0, 0.0], duration=0.3)
        
        time.sleep(0.5)
    
    # Take the photo!
    print("  📸 Click!")
    frame = mini.media.get_frame()
    
    # Success beep (higher pitch)
    success_beep = generate_beep(frequency=880, duration=0.4)
    mini.media.start_playing()
    for j in range(0, len(success_beep), 1024):
        mini.media.push_audio_sample(success_beep[j:j+1024])
    mini.media.stop_playing()
    
    # Celebration animation
    mini.goto_target(
        head=create_head_pose(pitch=-10, degrees=True),
        antennas=[0.5, -0.5],
        duration=0.5
    )
    
    # Save and display
    if frame is not None:
        filename = f"reachy_selfie_{int(time.time())}.jpg"
        cv2.imwrite(filename, frame)
        print(f"\n✓ Photo saved: {filename}")
        
        # Display
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 8))
        plt.imshow(frame_rgb)
        plt.title("Reachy's Selfie!")
        plt.axis('off')
        plt.show()
    
    # Return to neutral
    mini.goto_target(
        head=create_head_pose(),
        antennas=[0.0, 0.0],
        duration=1.0
    )

---

## 14. Exercises (Try It Yourself!)

### Exercise 1: Time-Lapse Photography

Create a program that:
1. Takes a photo every 5 seconds
2. Captures 5 photos total
3. Plays a beep before each photo
4. Saves all photos with sequential filenames

**Bonus:** Move the head slightly between each photo!

In [ ]:
# Your code here
with ReachyMini() as mini:
    # TODO: Implement time-lapse
    pass

### Exercise 2: Audio Echo Effect

Record 2 seconds of audio, then play it back twice in a row (like an echo).

**Hint:** You can use `np.concatenate()` to join two audio arrays.

In [ ]:
# Your code here
with ReachyMini() as mini:
    # TODO: Record and create echo effect
    pass

### Exercise 3: Motion-Triggered Selfie

Create a "security camera" that:
1. Monitors audio level continuously
2. When a loud sound is detected (high amplitude), takes a photo
3. Plays a beep to indicate photo was taken

**Hint:** Use `np.abs(audio_sample).max()` to detect loud sounds.

In [ ]:
# Your code here
AUDIO_THRESHOLD = 0.1  # Adjust this value based on your environment

with ReachyMini() as mini:
    # TODO: Implement motion-triggered camera
    pass

### Exercise 4: Musical Greeting

Create a greeting sequence that:
1. Plays a sequence of 3-4 tones (different frequencies)
2. Moves the head and antennas in sync with each tone
3. Takes a photo at the end

Make it musical and fun!

In [ ]:
# Your code here
# Try frequencies like: 262 (C), 330 (E), 392 (G), 523 (C)

with ReachyMini() as mini:
    # TODO: Create musical greeting
    pass

---

## 15. Media Backend Options

Reachy Mini supports different media backends for different use cases:

* **`"default"`**: Auto-detection based on hardware
  - Lite version: Uses OpenCV
  - Wireless version: Uses GStreamer (local) or WebRTC (remote)

* **`"gstreamer"`**: Force GStreamer backend
  - Better performance for video streaming
  - Requires GStreamer installation

* **`"webrtc"`**: Force WebRTC backend
  - Best for remote connections
  - Lower latency over network

* **`"no_media"`**: Disable all media
  - Faster connection
  - Use when camera/audio not needed

Example:
```python
with ReachyMini(media_backend="gstreamer") as mini:
    # Your code
```

---

## 16. Best Practices

### Camera:
* Check if `get_frame()` returns `None` before processing
* Remember: OpenCV uses BGR format, not RGB
* For real-time display outside Jupyter, use `cv2.imshow()` instead of matplotlib

### Audio:
* Always call `start_recording()` before `get_audio_sample()`
* Always call `stop_recording()` when done
* Similarly: `start_playing()` before `push_audio_sample()`, `stop_playing()` when done
* Audio samples are typically 16 kHz, mono or stereo
* Resample audio files if they don't match the expected sample rate

### Performance:
* Use `media_backend="no_media"` when you don't need camera/audio
* Don't poll `get_frame()` or `get_audio_sample()` too fast if you don't need real-time
* For high-performance applications, consider using threading

---

## 17. What's Next?

Congratulations! You've learned how to:
* ✅ Capture images from the camera
* ✅ Display and save photos
* ✅ Record audio from the microphone
* ✅ Play audio through the speaker
* ✅ Generate synthetic sounds
* ✅ Combine media with motion for interactive behaviors

In the next notebook, you'll dive deeper into motion control:
* 🎯 `goto_target` vs `set_target` - When to use each
* 🎨 Interpolation methods (linear, minjerk, ease, cartoon)
* 🔄 Continuous motion patterns
* ⏱️ Real-time control at high frequencies

➡️ **Next: Notebook 2 — Motion Control Deep Dive** 🎯

---

## Appendix: Quick Reference

### Camera
```python
# Capture frame
frame = mini.media.get_frame()  # Returns BGR numpy array

# Convert for display
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Save image
cv2.imwrite("photo.jpg", frame)
```

### Audio Recording
```python
# Start recording
mini.media.start_recording()

# Get samples
sample = mini.media.get_audio_sample()

# Stop recording
mini.media.stop_recording()

# Save to file
audio_data = np.concatenate(samples)
sf.write("audio.wav", audio_data, 16000)
```

### Audio Playback
```python
# Load audio
data, rate = sf.read("audio.wav", dtype='float32')

# Start playback
mini.media.start_playing()

# Push samples
for i in range(0, len(data), 1024):
    mini.media.push_audio_sample(data[i:i+1024])

# Stop playback
mini.media.stop_playing()
```

### Sample Rates
```python
input_rate = mini.media.get_input_audio_samplerate()
output_rate = mini.media.get_output_audio_samplerate()
```